In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox, ttk
import time

# ====================== Load Data ==========================
distance_df = pd.read_csv("C:/Users/Acer/OneDrive/Desktop/distance.csv")
#orders_df = pd.read_csv("C:/Users/Acer/OneDrive/Desktop/order_small.csv")
orders_df = pd.read_csv("C:/Users/Acer/OneDrive/Desktop/order_large.csv")

# Create city-to-index mappings
cities = pd.unique(distance_df[['Source', 'Destination']].values.ravel())
city_to_index = {city: idx for idx, city in enumerate(cities)}
index_to_city = {idx: city for city, idx in city_to_index.items()}

# Create distance matrix
n_cities = len(cities)
distance_matrix = np.full((n_cities, n_cities), np.inf)
for _, row in distance_df.iterrows():
    i = city_to_index[row['Source']]
    j = city_to_index[row['Destination']]
    d = row['Distance(M)']
    distance_matrix[i][j] = d
    distance_matrix[j][i] = d

# =================== Ant Colony Optimization ====================
class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=2):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.all_inds = range(len(distances))

    def run(self):
        shortest_path = None
        best = ([], np.inf)
        for _ in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromones(all_paths)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < best[1]:
                best = shortest_path
            self.pheromone *= (1 - self.decay)
        return best

    def spread_pheromones(self, paths):
        paths = sorted(paths, key=lambda x: x[1])
        for path, dist in paths[:self.n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / dist

    def gen_path(self, start):
        path = []
        visited = set([start])
        prev = start
        for _ in range(len(self.distances) - 1):
            move = self.pick_move(prev, visited)
            path.append((prev, move))
            visited.add(move)
            prev = move
        path.append((prev, start))
        return path

    def pick_move(self, current, visited):
        pheromone = np.copy(self.pheromone[current])
        pheromone[list(visited)] = 0
        heuristic = 1 / (self.distances[current] + 1e-10)
        heuristic[list(visited)] = 0
        prob = (pheromone ** self.alpha) * (heuristic ** self.beta)
        prob /= prob.sum()
        return np.random.choice(self.all_inds, 1, p=prob)[0]

    def gen_all_paths(self):
        paths = []
        for _ in range(self.n_ants):
            path = self.gen_path(0)
            dist = sum(self.distances[i][j] for i, j in path)
            paths.append((path, dist))
        return paths

# =================== Greedy Algorithm ======================
def greedy_route(matrix):
    visited = [0]
    total_distance = 0
    while len(visited) < len(matrix):
        last = visited[-1]
        next_city = np.argmin([matrix[last][j] if j not in visited else np.inf for j in range(len(matrix))])
        total_distance += matrix[last][next_city]
        visited.append(next_city)
    total_distance += matrix[visited[-1]][0]
    return visited, total_distance

# ========================= UI & Execution ==========================
def run_optimization():
    try:
        max_weight_kg = float(weight_entry.get())
        max_weight = max_weight_kg * 1000
        method = algo_choice.get()

        truck_id = 1
        current_weight = 0
        truck_assignments = {}
        current_group = []
        total_cost = 0
        total_distance = 0

        for _, row in orders_df.iterrows():
            weight = row['Weight']
            if current_weight + weight > max_weight:
                truck_assignments[f"Truck_{truck_id}"] = current_group
                truck_id += 1
                current_weight = 0
                current_group = []
            current_group.append(row)
            current_weight += weight

        if current_group:
            truck_assignments[f"Truck_{truck_id}"] = current_group

        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, f"\n🚚 Number of Trucks Used: {len(truck_assignments)}\n", 'title')
        output_text.insert(tk.END, "===============================\n", 'line')

        for truck, orders in truck_assignments.items():
            city_names = [row['Destination'] for row in orders if row['Destination'] in city_to_index]
            city_indices = [city_to_index[city] for city in city_names]
            if not city_indices:
                continue

            to_keep = [0] + city_indices
            submatrix = distance_matrix[np.ix_(to_keep, to_keep)]
            local_index_to_city = {i: index_to_city[j] for i, j in enumerate(to_keep)}

            start_time = time.time()

            if method == "ACO":
                aco = AntColony(
                    distances=submatrix,
                    n_ants=10, n_best=3, n_iterations=30, decay=0.1, alpha=1, beta=2
                )
                best_path, best_cost = aco.run()
                route = [local_index_to_city[i] for i, _ in best_path] + [local_index_to_city[best_path[-1][1]]]
            elif method == "Greedy":
                greedy_path, best_cost = greedy_route(submatrix)
                route = [local_index_to_city[i] for i in greedy_path]
                route.append(route[0])

            exec_time = time.time() - start_time

            total_weight = sum([r['Weight'] for r in orders])
            order_ids = [str(r['Order_ID']) for r in orders]

            output_text.insert(tk.END, f"\n🛻 {truck}\n", 'subtitle')
            output_text.insert(tk.END, f"   Orders: {', '.join(order_ids)}\n", 'text')
            output_text.insert(tk.END, f"   Total Weight: {total_weight/1000:.2f} kg\n", 'text')
            output_text.insert(tk.END, f"   Route: {' -> '.join(route)}\n", 'route')
            output_text.insert(tk.END, f"   Distance: {best_cost:.2f} meters\n", 'text')
            output_text.insert(tk.END, f"   Execution Time: {exec_time:.4f} seconds\n", 'text')

            total_distance += best_cost
            total_cost += best_cost * 0.5

        output_text.insert(tk.END, f"\n📏 Total Delivery Distance for All Trucks: {total_distance:.2f} meters\n", 'title')
        output_text.insert(tk.END, f"💰 Total Estimated Delivery Cost: {total_cost:.2f} currency units\n", 'title')

    except ValueError:
        messagebox.showerror("Input Error", "Please enter a valid number for weight")

# ===================== GUI Setup ========================
root = tk.Tk()
root.title("Vehicle Routing Problem Optimizer")
root.geometry("850x750")
root.configure(bg="#f0f4f8")

frame = tk.Frame(root, bg="#ffffff", padx=20, pady=20, relief=tk.RAISED, bd=2)
frame.pack(pady=20)

header = tk.Label(frame, text="🚚 VRP using ACO / Greedy", font=("Helvetica", 16, "bold"), bg="#ffffff", fg="#333399")
header.pack(pady=10)

weight_label = tk.Label(frame, text="Enter Max Truck Weight (kg):", bg="#ffffff", font=("Helvetica", 12))
weight_label.pack(anchor='w', pady=5)
weight_entry = tk.Entry(frame, font=("Helvetica", 12), bd=2, relief=tk.GROOVE)
weight_entry.pack(fill='x')

algo_label = tk.Label(frame, text="Choose Algorithm:", bg="#ffffff", font=("Helvetica", 12))
algo_label.pack(anchor='w', pady=5)
algo_choice = ttk.Combobox(frame, values=["ACO", "Greedy"], state="readonly", font=("Helvetica", 12))
algo_choice.current(0)
algo_choice.pack(fill='x')

run_button = tk.Button(frame, text="Run Optimization", command=run_optimization, bg="#333399", fg="white", font=("Helvetica", 14, "bold"), padx=10, pady=5)
run_button.pack(pady=15)

output_text = tk.Text(root, height=25, width=110, font=("Courier New", 11), bg="#e6e6e6")
output_text.pack(padx=10, pady=10)

# Text tags for formatting
output_text.tag_config('title', foreground="#1a237e", font=("Helvetica", 13, "bold"))
output_text.tag_config('line', foreground="#757575")
output_text.tag_config('subtitle', foreground="#303f9f", font=("Helvetica", 12, "bold"))
output_text.tag_config('text', foreground="#212121")
output_text.tag_config('route', foreground="#0d47a1", font=("Courier New", 10, "italic"))

root.mainloop()


C:\Users\Acer\AppData\Local\Temp\ipykernel_15064\3055192073.py:76: RuntimeWarning: invalid value encountered in divide
  prob /= prob.sum()
